<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/Session3_MarioRosas_LaurenTucker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 2 (Document structure) - ILTH

**Student:** Mario Rosas Otero

## Paraphrases Template

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# importing the "tarfile" module
import tarfile

def unzip_tars(root, files, targetFolder = None):
  for fileN in files:
    file = tarfile.open(root + fileN)# open file
    if targetFolder:
      file.extractall(targetFolder) # extracting file
    else:
      file.extractall(root) # extracting file
    file.close()

In [3]:
# unzip_tars('/content/drive/MyDrive/UPC Artificial Intelligence Master/IHLT/Complementary Material/', ['test-gold.tgz', 'task6-submissions.tgz', 'train.tgz', 'trial.tgz'], '/content/drive/MyDrive/UPC Artificial Intelligence Master/IHLT/Complementary Material/')

In [4]:
import pandas as pd

In [13]:
dt = pd.read_csv('/content/drive/My Drive/UPC/First Semester/IHLT - Introduction to Human Language Technology/Lab Sessions/Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [14]:
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [16]:
dt['gs'] = pd.read_csv('/content/drive/My Drive/UPC/First Semester/IHLT - Introduction to Human Language Technology/Lab Sessions/Complementary Material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

In [17]:
dt.shape

(459, 3)

In [18]:
dt.head()

,0,1,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


# Excercise

In [30]:
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from scipy.stats import pearsonr
import numpy as np

import string
import collections
import nltk
from nltk.corpus import gutenberg
from nltk.text import Text
nltk.download('gutenberg')
nltk.download('stopwords')

def remove_signs(wrd):
  wrd = list(wrd)
  wrd = [word for word in wrd if not any(caracter in signs for caracter in word)]
  wrd = ''.join(wrd)
  return wrd

def clean(corpus, stopwords, minwords_len, signs):
   corpus = corpus.split(' ')
   corpus = [word.lower() for word in corpus]
   corpus = [word if not any(caracter in signs for caracter in word) else remove_signs(word) for word in corpus]
   corpus = [word for word in corpus if word not in stopwords and word.isalpha()]
   corpus = [word for word in corpus if len(word) > minwords_len]
   return corpus

from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
pos_map = {'N': NOUN,
           'V':VERB,
           'J':ADJ,
           'R':ADV}

corpus = list(gutenberg.words('blake-poems.txt'))
stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
cleaned = clean(dt.loc[0][0], stopwords, minwords_len, signs)

In [21]:
pairs = nltk.pos_tag(cleaned)
pairs

[('leaders', 'NNS'),
 ('given', 'VBN'),
 ('new', 'JJ'),
 ('chance', 'NN'),
 ('let', 'VBD'),
 ('hope', 'PRP'),
 ('seize', 'VB')]

In [67]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
    if p[1][0] in {'N','V','J','R'}:
        return wnl.lemmatize(p[0].lower(), pos=pos_map[p[1][0]])
    return p[0]

def lemmatize_sentence(sentence):
  cleaned = clean(sentence, stopwords, minwords_len, signs)
  tagged = nltk.pos_tag(cleaned)
  return[lemmatize(pair) for pair in tagged]

# [lemmatize(pair) for pair in pairs]
lemmatize_sentence(dt.loc[0][0])

['leader', 'give', 'new', 'chance', 'let', 'hope', 'seize']

In [68]:
def compute_metric(metric, elements): # Fenction to calculate Jaccard Distance
  if metric == 'jaccard':
      res = jaccard_distance(set(lemmatize_sentence(elements[0])),
                             set(lemmatize_sentence(elements[1])))
  return res

In [69]:
dt['jaccard'] = ""

In [70]:
lemmatize_sentence(dt.loc[317][0]), lemmatize_sentence(dt.loc[317][1])

(['parliament', 'adopt', 'legislative', 'resolution'],
 ['parliament', 'adopt', 'legislative', 'resolution'])

In [71]:
clean(dt.iloc[455][0], stopwords, minwords_len, signs), clean(dt.iloc[455][1], stopwords, minwords_len, signs)

(['vote', 'take', 'place', 'today'], ['vote', 'take', 'place'])

In [72]:
clean(dt.iloc[455][0], stopwords, minwords_len, signs), clean(dt.iloc[455][1], stopwords, minwords_len, signs)

(['vote', 'take', 'place', 'today'], ['vote', 'take', 'place'])

In [73]:
dt

,0,1,gs,jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,
...,...,...,...,...
454,It is our job to continue to support Latvia wi...,It is of our duty of continue to support the c...,5.000,
455,The vote will take place today at 5.30 p.m.,Vote will take place at 17 h 30.,4.750,
456,Neither was there a qualified majority within ...,There was no qualified majority in this Parlia...,5.000,
457,Let me remind you that our allies include ferv...,"I hold you recall that our allies, there are e...",4.000,


In [74]:
for i in range(len(dt)): # Iterating to get Jaccard distances over each column of the dataframe
  dt['jaccard'][i] = np.float64(compute_metric('jaccard',[dt.iloc[i][0],dt.iloc[i][1]]))*10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
dt

,0,1,gs,jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,6.666667
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,0.0
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,7.272727
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,2.5
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,0.0
...,...,...,...,...
454,It is our job to continue to support Latvia wi...,It is of our duty of continue to support the c...,5.000,6.363636
455,The vote will take place today at 5.30 p.m.,Vote will take place at 17 h 30.,4.750,2.5
456,Neither was there a qualified majority within ...,There was no qualified majority in this Parlia...,5.000,6.666667
457,Let me remind you that our allies include ferv...,"I hold you recall that our allies, there are e...",4.000,8.0


In [76]:
pearsonr(dt['gs'], 1-dt['jaccard']) # Calculating the pearson correlation between GS results and 1-Jaccard calculated dats

(0.4810231734170826, 5.9045104154977e-28)

In [77]:
lemmatize_sentence(dt.iloc[454][0]), lemmatize_sentence(dt.iloc[454][1])

(['job',
  'continue',
  'support',
  'latvia',
  'integration',
  'russian',
  'population'],
 ['duty',
  'continue',
  'support',
  'country',
  'issue',
  'integration',
  'russian',
  'people'])

# Conclusion



